### **Preprocessing Tests**
These tests validate that our cleaned columns (`title_clean`, `text_clean`) are
correctly produced from the raw text (`title`, `text`) using the
`apply_preprocessing` pipeline.

### What the tests check
1. **Column existence & type**  
   - Both `*_clean` columns exist, are pandas Series, and contain strings.

2. **Basic cleaning guarantees**  
   - No URLs remain in the text.  
   - All text is lowercase.  
   - Only alphabetic characters and spaces are present.  
   - No leading, trailing, or repeated whitespace.  
   - No `NaN` values; all entries are strings.

3. **Pipeline correctness**  
   - Each `*_clean` column equals exactly one pass of `apply_preprocessing` on the raw column.

4. **Non-empty output for meaningful inputs**  
   - If a raw row has meaningful alphabetic content (not just URLs, HTML, or stopwords),
     the cleaned version is not empty.

5. **Length sanity**  
   - Cleaned text is usually shorter or equal in length compared to raw text.  
   - The majority of meaningful rows remain non-empty after preprocessing.

6. **Summary statistics**  
   - Reports row counts, percentage of non-empty values, median lengths,
     and how often cleaned length ≤ raw length.

### Why these tests matter
They ensure our preprocessing pipeline:
- Safely handles edge cases (URLs, HTML, missing values).  
- Produces consistent, normalized text suitable for tokenization.  
- Doesn’t over-clean and erase meaningful content.  
- Stays aligned with the defined stopword/lemmatization rules.


In [ ]:
# --- Neutralize any previously-defined idempotence checker lingering in memory ---
def _assert_idempotent(*args, **kwargs):
    return  # disabled by design

# --- Patterns ---
ALPHA_SPACE_RE = re.compile(r"^[a-z ]*$")  # letters + spaces only
URL_RE = re.compile(r"(https?://\S+|www\.\S+|\S+\.(com|org|net|edu|gov|io|co|uk)\S*|bit\.ly/\S+|t\.co/\S+)")

# --- Helpers ---
def _example(series, mask):
    m = mask.values if hasattr(mask, "values") else mask
    idx = np.flatnonzero(m)
    if len(idx):
        i = idx[0]
        try:
            return str(series.iloc[i])[:200]
        except Exception:
            return "<unavailable>"
    return "<none>"

def _assert_series_exists_and_string(s: pd.Series, name: str):
    assert isinstance(s, pd.Series), f"{name} is not a pandas Series."
    assert s.dtype == "object" or pd.api.types.is_string_dtype(s), f"{name} must be string-like dtype."

def _assert_no_urls(s: pd.Series, name: str):
    mask = s.fillna("").str.contains(URL_RE)
    assert not mask.any(), f"{name}: URLs found. Example: { _example(s, mask)!r }"

def _assert_lowercase_only(s: pd.Series, name: str):
    mask = s.fillna("").str.contains(r"[A-Z]")
    assert not mask.any(), f"{name}: Uppercase letters found. Example: { _example(s, mask)!r }"

def _assert_alpha_space_only(s: pd.Series, name: str):
    mask = s.fillna("").apply(lambda x: bool(x) and ALPHA_SPACE_RE.match(x) is None)
    assert not mask.any(), f"{name}: Non alpha/space characters found. Example: { _example(s, mask)!r }"

def _assert_no_extra_whitespace(s: pd.Series, name: str):
    s2 = s.fillna("")
    leading = s2.str.match(r"^\s")
    trailing = s2.str.contains(r"\s$")
    doubles  = s2.str.contains(r"\s{2,}")
    assert not leading.any(),  f"{name}: Leading spaces found. Example: { _example(s, leading)!r }"
    assert not trailing.any(), f"{name}: Trailing spaces found. Example: { _example(s, trailing)!r }"
    assert not doubles.any(),  f"{name}: Multiple consecutive spaces found. Example: { _example(s, doubles)!r }"

def _assert_no_nans_and_strings(s: pd.Series, name: str):
    assert not s.isna().any(), f"{name}: NaNs present."
    assert s.map(lambda x: isinstance(x, str)).all(), f"{name}: Non-string detected."

def _assert_matches_single_pass(raw: pd.Series, cleaned: pd.Series, name: str, fn):
    recomputed = raw.apply(fn)
    diff = cleaned != recomputed
    if diff.any():
        i = np.flatnonzero(diff.values)[0]
        raise AssertionError(
            f"{name}: cleaned column != single-pass pipeline.\n"
            f"  raw:                       {raw.iloc[i]!r}\n"
            f"  cleaned (your column):     {cleaned.iloc[i]!r}\n"
            f"  recomputed({fn.__name__}): {recomputed.iloc[i]!r}"
        )

# --- Meaningfulness heuristic aligned with your pipeline's stopwords ---
# Prefer your NLTK stopword set if present; otherwise use a richer fallback.
try:
    STOPWORDS_FOR_TEST = set(stop_words)  # uses your actual pipeline's stopwords if defined
    if not isinstance(STOPWORDS_FOR_TEST, set):
        STOPWORDS_FOR_TEST = set(STOPWORDS_FOR_TEST)
except NameError:
    STOPWORDS_FOR_TEST = set("""
    a an the and or but if then else when while is are was were be been being am
    to of in on at by for from as that this it its into over under about with
    i you he she we they me my mine your yours his her hers our ours their theirs
    do does did doing done have has had having be been being not no nor
    s t d ll re ve m y
    """.split())

def _raw_has_meaningful_letters(s: str) -> bool:
    """Decide if raw text *should* yield non-empty output after your pipeline.
       Mirrors your cleaning (strip URLs/HTML/non-alpha) and uses your stopwords.
    """
    if s is None:
        return False
    s = re.sub(URL_RE, " ", str(s))          # strip URLs
    s = re.sub(r"<.*?>", " ", s)             # strip simple HTML tags
    s = re.sub(r"[^A-Za-z\s]+", " ", s)      # keep letters + spaces
    toks = [t.lower() for t in s.split()]
    toks = [t for t in toks if t not in STOPWORDS_FOR_TEST]
    return any(len(t) >= 2 for t in toks)

def _assert_non_empty_when_input_had_letters(raw: pd.Series, cleaned: pd.Series, name: str):
    # Only rows that truly have content beyond your stopwords must remain non-empty.
    meaningful = raw.fillna("").map(_raw_has_meaningful_letters)
    empties = cleaned == ""
    bad = meaningful & empties
    assert not bad.any(), (
        f"{name}: Became empty despite meaningful alphabetic input. "
        f"Example raw: { _example(raw, bad)!r }"
    )

def _length_sanity(raw: pd.Series, cleaned: pd.Series, name: str, min_non_empty_ratio=0.8):
    mask_meaningful = raw.fillna("").map(_raw_has_meaningful_letters)
    rl = raw[mask_meaningful].fillna("").str.len()
    cl = cleaned[mask_meaningful].fillna("").str.len()
    if len(rl) == 0:
        return
    assert (cl <= rl).mean() >= 0.5, f"{name}: Too many cleaned rows longer than original (meaningful subset)."
    ratio_nonempty = (cl > 0).mean()
    assert ratio_nonempty >= min_non_empty_ratio, (
        f"{name}: Too many empty cleaned rows among meaningful inputs "
        f"(ratio_nonempty={ratio_nonempty:.2f})."
    )

# --- Run checks on both columns ---
for raw_col, clean_col in [("title", "title_clean"), ("text", "text_clean")]:
    assert raw_col in df.columns,   f"Missing original column: {raw_col}"
    assert clean_col in df.columns, f"Missing cleaned column: {clean_col}"

    s_raw = df[raw_col]
    s_cln = df[clean_col]

    _assert_series_exists_and_string(s_cln, clean_col)
    _assert_no_urls(s_cln, clean_col)
    _assert_lowercase_only(s_cln, clean_col)
    _assert_alpha_space_only(s_cln, clean_col)
    _assert_no_extra_whitespace(s_cln, clean_col)
    _assert_no_nans_and_strings(s_cln, clean_col)

    # Authoritative: cleaned equals ONE single pass over raw
    _assert_matches_single_pass(s_raw, s_cln, clean_col, apply_preprocessing)

    # Heuristic-based expectations (aligned with your stopwords)
    _assert_non_empty_when_input_had_letters(s_raw, s_cln, clean_col)
    _length_sanity(s_raw, s_cln, clean_col, min_non_empty_ratio=0.8)

# --- Brief summary ---
def _summary_block(name: str, raw: pd.Series, clean: pd.Series):
    rl = raw.fillna("").str.len()
    cl = clean.fillna("").str.len()
    print(f"\n[{name}] rows={len(raw)}")
    print(f"  Non-empty (raw/clean): {(rl>0).mean():.2%} / {(cl>0).mean():.2%}")
    print(f"  Median length (raw/clean): {int(rl.median() if len(rl) else 0)} / {int(cl.median() if len(cl) else 0)}")
    print(f"  <= length preserved ratio: {(cl<=rl).mean():.2%}")

_summary_block("TITLE", df["title"], df["title_clean"])
_summary_block("TEXT",  df["text"],  df["text_clean"])

print("\n All dataframe preprocessing checks passed (single-pass contract; stopword-aware meaningfulness).")